In [24]:
from backbone import ResNet
import matplotlib.pyplot as plt
import numpy as np
from region_proposal import RPN, ProposalLayer
from resnet_var_dict import resnet_var_dict
import skimage
from skimage import io, transform
import tensorflow as tf
from utils import generate_anchors

In [2]:
tf.enable_eager_execution()

In [3]:
backbone = ResNet('channels_last', num_blocks=[3, 4, 6, 3], include_fc=False)
rpn_graph = RPN('channels_last')
proposer = ProposalLayer()

In [4]:
real_picture_1 = io.imread('./example_data/imagenet_tabby_cat_1.jpg').astype('float32')
real_picture_2 = io.imread('./example_data/imagenet_tabby_cat_3.jpg').astype('float32')
input_data = tf.convert_to_tensor(np.stack([real_picture_1, real_picture_2]))

In [5]:
feature_maps = backbone(input_data)
rpn_logits, rpn_probs, rpn_boxes = rpn_graph(feature_maps)

In [54]:
anchors = generate_anchors([32, 64, 128], [0.5, 1, 1.5], 
                           tf.shape(input_data)[1:3],
                           tf.shape(feature_maps)[1:3], 1)
anchors = tf.tile(tf.expand_dims(anchors,0), [tf.shape(input_data)[0], 1, 1])

In [96]:
rois = proposer([rpn_probs, rpn_boxes, anchors])